# Analis exploratorio para libros de programas de catedra
Este analisis surge por la necesidad de comparar el material de la catedra, que figura en los programas, con respecto a los datos que me ortoga la bilbioteca. ¿Que diferencia hay entre los libros ya procesados? y es que ese procesamiento se hizo pensando en que la medida a comparar sea el valos  "ISBN" de cada libro, en este caso se piensa que la medida sera solo el autor y libro.

## Librerias y ruta de acceso

In [1]:
import sys
import os

sys.path.append(os.path.abspath('C:/Users/Axel/Desktop/Practica-Biblioteca'))

from src import lectura, conversor
import pandas as pd

ISBN-10: 846044757X -> ISBN-13: 9788460447573


In [2]:
#Lectura de los 3 archivos
df1 = lectura.leer_csv('C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/bidi.csv')
df2 = lectura.leer_csv('C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/consulta_koha.csv')
df3 = lectura.leer_csv('C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/total_koha.csv')

Archivo C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/bidi.csv leído con éxito.
Archivo C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/consulta_koha.csv leído con éxito.
Archivo C:/Users/Axel/Desktop/Practica-Biblioteca/data/crudos/total_koha.csv leído con éxito.


In [3]:
# del df1 se toman las columnas 1 y 2
df1 = df1.iloc[:, 1:3]
df1.columns = ['title', 'author']

In [4]:
#del df2 y df3 se toman las columnas 'title' y 'author'
df2 = df2[['title', 'author']]
df3 = df3[['title', 'author']]

In [5]:
#en la columna titulo se encuentran titulos y ediciones o volumenes, se procede a solo dejar los titulos
# en el caso donde dice edicion se presenta asi " administración de ventas (8a edición)" donde el titulo es "administración de ventas" 
# y la edicion es "8a edición" entre parentesis se procede a eliminar la edicion

df1['title'] = df1['title'].str.replace(r'\(.*?\)', '', regex=True).str.strip() # Eliminar ediciones 

#para el caso de los volumenes se presenta asi "química orgánica: volumen 1" o "fisica preuniversitaria. volumen i" es decir el separados es dos puntos o uno
#se procede a eliminar los volumenes

import re

def limpiar_titulo(title):
    # Expresión regular para detectar "volumen" o "vol." con su número o identificación
    patron_volumen = r'(?i)(?:\bvol(?:umen)?\.?\s*[ivxlcdm]+\b|\bvolumen\s*\d+\b)'
    
    # Buscar cualquier ocurrencia de "volumen" o "vol." y dividir el string antes de eso
    title_limpio = re.split(patron_volumen, title)[0]
    
    # Eliminar espacios en blanco adicionales al inicio y al final
    return title_limpio.strip()

# Aplicar la función a la columna 'title'
df1['title'] = df1['title'].apply(limpiar_titulo)

In [6]:
df2 = df2.dropna()
df3 = df3.dropna()
df2 = df2.drop_duplicates()
df3 = df3.drop_duplicates()

df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1209 entries, 0 to 1547
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1209 non-null   object
 1   author  1209 non-null   object
dtypes: object(2)
memory usage: 28.3+ KB


In [7]:
#Agregar columna de origen
df1['origen'] = 'bidi'
df2['origen'] = 'consulta_koha'
df3['origen'] = 'total_koha'
#juntar los 3 dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9449 entries, 0 to 9448
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   9449 non-null   object
 1   author  9447 non-null   object
 2   origen  9449 non-null   object
dtypes: object(3)
memory usage: 221.6+ KB


In [9]:
# cambiamos los nombres de las columnas que estan en ingles a español
df.rename(columns={'title':'titulo', 'author':'autor'}, inplace=True)
df['titulo'] = df['titulo'].str.lower().str.strip()
df['autor'] = df['autor'].str.lower().str.strip()

# hay titulos que tienen ":" o "/" al final de la cadena, se eliminan
df['titulo'] = df['titulo'].str.replace(':', '').str.replace('/', '').str.replace('.', '')

# Eliminar ':' o '/' o '.' solo al final de la cadena
df['autor'] = df['autor'].str.replace(r'[:/.]+$', '', regex=True)

df['titulo'] = df['titulo'].str.lower().str.strip()

In [10]:
df.head(50)

,titulo,autor,origen
0,la libertad como tecnología de gobierno derech...,beatriz dávilo [et al],bidi
1,semillas de orquídeas,víctor h. lallana (compilador),bidi
2,breve manual de lectura y escritura para la u...,carolina clerici,bidi
3,manual de cálculos para el diseño de plantas d...,elbio miguel woeffray,bidi
4,revolución y lucha por la organización primera...,juan a. vilar,bidi
5,la confederación argentina época de rosas 1829...,juan a. vilar,bidi
6,hacia la derrota federal la confederación arge...,juan a. vilar,bidi
7,variaciones del estudiar desde relatos de estu...,maría amelia migueles [et al],bidi
8,política y felicidad análisis fragmentario de ...,gladis ana catalina martínez,bidi
9,la escuela secundaria disputa sentidos,carina rattero y candela san román (compiladoras),bidi


In [11]:
#escritura del archivo
df.to_csv('C:/Users/Axel/Desktop/Practica-Biblioteca/data/procesados/titulos_autores_biblioteca.csv', index=False)